In [ ]:
# 导入所需模块
try:
    import pytest  # type: ignore
    PYTEST_AVAILABLE = True
except ImportError:
    PYTEST_AVAILABLE = False
    print("注意：pytest 未安装，请运行 'pip install pytest' 安装")

import asyncio
from unittest.mock import Mock, patch, MagicMock
from typing import List, Dict, Any
import tempfile
import os

print("Pytest 测试环境设置完成！")

# 创建一些示例类和函数用于测试
class Calculator:
    """计算器类，用于演示测试"""
    
    def add(self, a: int, b: int) -> int:
        return a + b
    
    def divide(self, a: int, b: int) -> float:
        if b == 0:
            raise ValueError("除数不能为零")
        return a / b
    
    def multiply(self, a: int, b: int) -> int:
        return a * b

class UserService:
    """用户服务类，用于演示模拟测试"""
    
    def __init__(self, database):
        self.database = database
    
    def get_user(self, user_id: int) -> Dict[str, Any]:
        return self.database.find_user(user_id)
    
    def create_user(self, name: str, email: str) -> Dict[str, Any]:
        user_data = {"name": name, "email": email}
        return self.database.save_user(user_data)

print("示例类创建完成！")


In [ ]:
# 基础测试示例

# 1. 简单的测试函数
def test_calculator_add():
    """测试计算器的加法功能"""
    calc = Calculator()
    result = calc.add(2, 3)
    assert result == 5
    
    # 测试负数
    result = calc.add(-1, 1)
    assert result == 0
    
    # 测试零
    result = calc.add(0, 0)
    assert result == 0

def test_calculator_multiply():
    """测试计算器的乘法功能"""
    calc = Calculator()
    assert calc.multiply(3, 4) == 12
    assert calc.multiply(-2, 5) == -10
    assert calc.multiply(0, 100) == 0

def test_calculator_divide():
    """测试计算器的除法功能"""
    calc = Calculator()
    assert calc.divide(10, 2) == 5.0
    assert calc.divide(7, 2) == 3.5
    
    # 测试除零异常
    try:
        calc.divide(10, 0)
        assert False, "应该抛出 ValueError"
    except ValueError as e:
        assert str(e) == "除数不能为零"

# 2. 使用断言进行测试
def test_list_operations():
    """测试列表操作"""
    my_list = [1, 2, 3]
    
    # 测试长度
    assert len(my_list) == 3
    
    # 测试包含
    assert 2 in my_list
    assert 4 not in my_list
    
    # 测试添加
    my_list.append(4)
    assert my_list[-1] == 4
    
    # 测试删除
    my_list.remove(2)
    assert 2 not in my_list

# 3. 字符串测试
def test_string_operations():
    """测试字符串操作"""
    text = "Hello, World!"
    
    assert text.startswith("Hello")
    assert text.endswith("World!")
    assert "World" in text
    assert len(text) == 13
    
    # 测试大小写转换
    assert text.lower() == "hello, world!"
    assert text.upper() == "HELLO, WORLD!"

# 运行测试（在notebook中模拟pytest行为）
def run_test(test_func, test_name):
    """运行单个测试函数"""
    try:
        test_func()
        print(f"✓ {test_name} 通过")
        return True
    except Exception as e:
        print(f"✗ {test_name} 失败: {e}")
        return False

# 模拟运行测试
print("=== 基础测试演示 ===")
tests = [
    (test_calculator_add, "test_calculator_add"),
    (test_calculator_multiply, "test_calculator_multiply"),
    (test_calculator_divide, "test_calculator_divide"),
    (test_list_operations, "test_list_operations"),
    (test_string_operations, "test_string_operations"),
]

passed = 0
total = len(tests)

for test_func, test_name in tests:
    if run_test(test_func, test_name):
        passed += 1

print(f"\n测试结果: {passed}/{total} 通过")

# 如果pytest可用，展示如何使用
if PYTEST_AVAILABLE:
    print("\n注意：在实际项目中，使用 'pytest' 命令运行测试文件")


In [ ]:
# 测试夹具示例

# 1. 简单夹具
def setup_calculator():
    """设置计算器夹具"""
    print("设置计算器...")
    return Calculator()

def teardown_calculator(calc):
    """清理计算器夹具"""
    print("清理计算器...")
    del calc

# 2. 上下文管理器夹具
from contextlib import contextmanager

@contextmanager
def calculator_fixture():
    """计算器夹具上下文管理器"""
    print("设置计算器夹具...")
    calc = Calculator()
    try:
        yield calc
    finally:
        print("清理计算器夹具...")
        del calc

# 3. 数据库模拟夹具
@contextmanager
def mock_database():
    """模拟数据库夹具"""
    print("创建模拟数据库...")
    database = {
        "users": [
            {"id": 1, "name": "Alice", "email": "alice@example.com"},
            {"id": 2, "name": "Bob", "email": "bob@example.com"}
        ]
    }
    
    # 模拟数据库操作
    class MockDB:
        def __init__(self, data):
            self.data = data
        
        def find_user(self, user_id):
            for user in self.data["users"]:
                if user["id"] == user_id:
                    return user
            return None
        
        def save_user(self, user_data):
            user_id = max(user["id"] for user in self.data["users"]) + 1
            user_data["id"] = user_id
            self.data["users"].append(user_data)
            return user_data
    
    db = MockDB(database)
    try:
        yield db
    finally:
        print("清理模拟数据库...")

# 4. 临时文件夹具
@contextmanager
def temp_file_fixture():
    """临时文件夹具"""
    print("创建临时文件...")
    temp_file = tempfile.NamedTemporaryFile(mode='w+', delete=False)
    try:
        yield temp_file
    finally:
        print("清理临时文件...")
        temp_file.close()
        os.unlink(temp_file.name)

# 使用夹具进行测试
def test_calculator_with_fixture():
    """使用夹具测试计算器"""
    with calculator_fixture() as calc:
        assert calc.add(2, 3) == 5
        assert calc.multiply(4, 5) == 20
        
        # 测试除法异常
        try:
            calc.divide(10, 0)
            assert False, "应该抛出 ValueError"
        except ValueError:
            pass

def test_user_service_with_mock_db():
    """使用模拟数据库测试用户服务"""
    with mock_database() as db:
        user_service = UserService(db)
        
        # 测试获取用户
        user = user_service.get_user(1)
        assert user["name"] == "Alice"
        assert user["email"] == "alice@example.com"
        
        # 测试创建用户
        new_user = user_service.create_user("Charlie", "charlie@example.com")
        assert new_user["name"] == "Charlie"
        assert new_user["email"] == "charlie@example.com"
        assert "id" in new_user

def test_file_operations():
    """测试文件操作"""
    with temp_file_fixture() as temp_file:
        # 写入测试数据
        test_data = "Hello, World!"
        temp_file.write(test_data)
        temp_file.flush()
        
        # 读取并验证
        temp_file.seek(0)
        read_data = temp_file.read()
        assert read_data == test_data

# 5. 参数化测试夹具
test_cases = [
    (2, 3, 5),
    (10, 15, 25),
    (-5, 8, 3),
    (0, 100, 100)
]

def test_calculator_add_parametrized():
    """参数化测试计算器加法"""
    with calculator_fixture() as calc:
        for a, b, expected in test_cases:
            result = calc.add(a, b)
            assert result == expected, f"期望 {a} + {b} = {expected}, 但得到 {result}"

# 运行夹具测试
print("=== 测试夹具演示 ===")

fixture_tests = [
    test_calculator_with_fixture,
    test_user_service_with_mock_db,
    test_file_operations,
    test_calculator_add_parametrized
]

for test_func in fixture_tests:
    test_name = test_func.__name__
    try:
        test_func()
        print(f"✓ {test_name} 通过")
    except Exception as e:
        print(f"✗ {test_name} 失败: {e}")

print("\n测试夹具演示完成")


In [ ]:
# 模拟和Mock测试示例

# 1. 简单Mock示例
def test_user_service_with_mock():
    """使用Mock测试用户服务"""
    # 创建Mock数据库
    mock_db = Mock()
    mock_db.find_user.return_value = {"id": 1, "name": "Alice", "email": "alice@example.com"}
    
    # 创建用户服务
    user_service = UserService(mock_db)
    
    # 测试获取用户
    user = user_service.get_user(1)
    
    # 验证结果
    assert user["name"] == "Alice"
    assert user["email"] == "alice@example.com"
    
    # 验证Mock被正确调用
    mock_db.find_user.assert_called_once_with(1)
    print("✓ Mock测试通过")

# 2. Mock方法的副作用
def test_mock_side_effects():
    """测试Mock的副作用"""
    mock_db = Mock()
    
    # 设置不同调用的不同返回值
    mock_db.find_user.side_effect = [
        {"id": 1, "name": "Alice"},
        {"id": 2, "name": "Bob"},
        None  # 第三次调用返回None
    ]
    
    user_service = UserService(mock_db)
    
    # 测试多次调用
    user1 = user_service.get_user(1)
    user2 = user_service.get_user(2)
    user3 = user_service.get_user(3)
    
    assert user1["name"] == "Alice"
    assert user2["name"] == "Bob"
    assert user3 is None
    
    # 验证调用次数
    assert mock_db.find_user.call_count == 3
    print("✓ Mock副作用测试通过")

# 3. Mock异常
def test_mock_exceptions():
    """测试Mock抛出异常"""
    mock_db = Mock()
    mock_db.find_user.side_effect = Exception("数据库连接失败")
    
    user_service = UserService(mock_db)
    
    try:
        user_service.get_user(1)
        assert False, "应该抛出异常"
    except Exception as e:
        assert str(e) == "数据库连接失败"
        print("✓ Mock异常测试通过")

# 4. 使用patch装饰器
@patch('time.sleep')
def test_function_with_sleep(mock_sleep):
    """测试包含sleep的函数"""
    import time
    def slow_function():
        time.sleep(1)
        return "完成"
    
    result = slow_function()
    
    # 验证sleep被调用但实际不会等待
    mock_sleep.assert_called_once_with(1)
    assert result == "完成"
    print("✓ patch装饰器测试通过")

# 5. 上下文管理器patch
def test_with_context_manager_patch():
    """使用上下文管理器patch"""
    def network_request(url):
        # 模拟网络请求
        import requests
        response = requests.get(url)
        return response.json()
    
    with patch('requests.get') as mock_get:
        # 配置Mock返回值
        mock_response = Mock()
        mock_response.json.return_value = {"status": "success", "data": "test"}
        mock_get.return_value = mock_response
        
        # 测试函数
        result = network_request("https://api.example.com/data")
        
        # 验证结果
        assert result["status"] == "success"
        assert result["data"] == "test"
        
        # 验证Mock被调用
        mock_get.assert_called_once_with("https://api.example.com/data")
        print("✓ 上下文管理器patch测试通过")

# 6. MagicMock示例
def test_magic_mock():
    """测试MagicMock"""
    mock_obj = MagicMock()
    
    # MagicMock支持魔术方法
    mock_obj.__len__.return_value = 5
    mock_obj.__getitem__.return_value = "test_value"
    
    # 测试魔术方法
    assert len(mock_obj) == 5
    assert mock_obj[0] == "test_value"
    
    # 验证调用
    mock_obj.__len__.assert_called_once()
    mock_obj.__getitem__.assert_called_once_with(0)
    print("✓ MagicMock测试通过")

# 7. 属性Mock
def test_property_mock():
    """测试属性Mock"""
    class MockUser:
        def __init__(self):
            self.name = "Test User"
            self.email = "test@example.com"
    
    mock_user = Mock(spec=MockUser)
    mock_user.name = "Alice"
    mock_user.email = "alice@example.com"
    
    # 测试属性
    assert mock_user.name == "Alice"
    assert mock_user.email == "alice@example.com"
    print("✓ 属性Mock测试通过")

# 8. 参数匹配
def test_call_arguments():
    """测试调用参数匹配"""
    mock_func = Mock()
    
    # 调用函数
    mock_func("arg1", "arg2", keyword="value")
    mock_func(42, keyword="test")
    
    # 验证特定调用
    mock_func.assert_any_call("arg1", "arg2", keyword="value")
    mock_func.assert_any_call(42, keyword="test")
    
    # 验证调用次数
    assert mock_func.call_count == 2
    
    # 检查所有调用
    calls = mock_func.call_args_list
    assert len(calls) == 2
    print("✓ 参数匹配测试通过")

# 9. 异步Mock
async def test_async_mock():
    """测试异步Mock"""
    mock_async_func = Mock()
    mock_async_func.return_value = asyncio.Future()
    mock_async_func.return_value.set_result("异步结果")
    
    # 测试异步调用
    result = await mock_async_func()
    assert result == "异步结果"
    
    mock_async_func.assert_called_once()
    print("✓ 异步Mock测试通过")

# 10. 综合测试示例
class EmailService:
    def __init__(self, smtp_client):
        self.smtp_client = smtp_client
    
    def send_email(self, to: str, subject: str, body: str):
        # 验证邮箱格式
        if "@" not in to:
            raise ValueError("无效的邮箱地址")
        
        # 发送邮件
        self.smtp_client.send_message(to, subject, body)
        return {"status": "sent", "to": to}

def test_email_service_integration():
    """综合测试邮件服务"""
    # 创建Mock SMTP客户端
    mock_smtp = Mock()
    email_service = EmailService(mock_smtp)
    
    # 测试成功发送
    result = email_service.send_email("test@example.com", "测试", "测试内容")
    
    # 验证结果
    assert result["status"] == "sent"
    assert result["to"] == "test@example.com"
    
    # 验证Mock被正确调用
    mock_smtp.send_message.assert_called_once_with("test@example.com", "测试", "测试内容")
    
    # 测试无效邮箱
    try:
        email_service.send_email("invalid-email", "测试", "内容")
        assert False, "应该抛出异常"
    except ValueError as e:
        assert str(e) == "无效的邮箱地址"
    
    print("✓ 邮件服务综合测试通过")

# 运行Mock测试
print("=== 模拟和Mock测试演示 ===")

mock_tests = [
    test_user_service_with_mock,
    test_mock_side_effects,
    test_mock_exceptions,
    test_function_with_sleep,
    test_with_context_manager_patch,
    test_magic_mock,
    test_property_mock,
    test_call_arguments,
    test_email_service_integration
]

for test_func in mock_tests:
    try:
        test_func()
    except Exception as e:
        print(f"✗ {test_func.__name__} 失败: {e}")

# 异步测试
async def run_async_tests():
    await test_async_mock()

try:
    await run_async_tests()
except Exception as e:
    print(f"✗ 异步测试失败: {e}")

print("\n=== 测试总结 ===")
print("""
测试最佳实践：
1. 使用describe性的测试函数名
2. 遵循Arrange-Act-Assert模式
3. 使用Mock隔离外部依赖
4. 编写独立的测试用例
5. 使用参数化测试覆盖多种场景
6. 测试异常情况和边界条件
7. 保持测试简单和可读
8. 使用夹具管理测试数据
""")
